LIBRARIES

In [ ]:
import sys,logging,time,os.path,math
from scipy.optimize import curve_fit
import numpy as np
from datetime import datetime
from pathlib import Path

STARTING MESSAGES

In [ ]:
start = time.time()
stloc = time.localtime(start)
logging.basicConfig(format='%(levelname)-7s    %(message)s',level=9)
logging.info("Start at time: {}:{}:{} of {}/{}/{}.".format(stloc[3],stloc[4],stloc[5],stloc[2],stloc[1],stloc[0]))

READING FXs

In [ ]:
def pdbAtom(a):
    ##01234567890123456789012345678901234567890123456789012345678901234567890123456789
    ##ATOM   2155 HH11 ARG C 203     116.140  48.800   6.280  1.00  0.00
    if a.startswith("TER"):
        return 0
    # NOTE: The 27th field of an ATOM line in the PDB definition can contain an
    #       insertion code. We shift that 20 bits and add it to the residue number
    #       to ensure that residue numbers will be unique.
    ## ===> atom name,       res name,        res id,                        chain,
    return (a[12:16].strip(),a[17:20].strip(),int(a[22:26])+(ord(a[26])<<20),a[21],
    ##            x,              y,              z, atom number
    float(a[30:38]),float(a[38:46]),float(a[46:54]),int(a[6:11])) #7:12


# Function for splitting a PDB file in chains, based on chain identifiers and TER statements
def pdbChains(pdbAtomList):
    chain = []
    for atom in pdbAtomList:
        if not atom: # Was a "TER" statement
            if chain:
                yield chain
            else:
                logging.info("Skipping empty chain definition")
            chain = [] 
            continue
        if not chain or chain[-1][3] == atom[3]:
            chain.append(atom)
        else:
            yield chain
            chain = [atom]
    if chain:
        yield chain

# PDB iterator v2
def pdbFrameIterator(streamIterator):
    res_list = ["HOH","SOL","TIP","WAT","CL","NA","ZN","Cl","Cl-","Na","Na+","Zn"]
    title, atoms = [], []
    for i in streamIterator:
        if i.startswith("ENDMDL"): 
            yield "".join(title), atoms #probably useless with 1 model per pdb
            title, atoms = [], []         
        elif i.startswith("TITLE"):
            title.append(i)
        elif i.startswith("HETATM"):
            logging.info('Skipping: %s' %i.rstrip("\n"))
        elif ((i.startswith("ATOM") or i.startswith("TER")) and not (i[17:20].strip().startswith(tuple(res_list)))):
            atoms.append(pdbAtom(i))      
    if atoms:
        yield "".join(title), atoms

# PDB stream generator
def streamTag(stream):

    if type(stream) == str:
        logging.info('Read input structure from file %s.' %stream)
        s = open(stream)

    for i in s:
        yield i

# Residues check
def residues(atomList):
    residue = [atomList[0]]
    for atom in atomList[1:]:
        if (atom[1] == residue[-1][1] and # Residue name check
            atom[2] == residue[-1][2] and # Residue id check
            atom[3] == residue[-1][3]):   # Chain id check
            residue.append(atom)
        else:
            yield Residue(residue)
            residue = [atom]
    yield Residue(residue)

# Split a string                                                              
def spl(x):                                                                   
    return x.split()

# Make a dictionary from two lists                                            
def hash(x,y):                                                                
    return dict(zip(x,y))

# CHECK INPUT FOLDER, READ FILE NAMES FROM INPUT FOLDER, SORT BY CREATION DATE, STORE THE LIST INTO A FILE TO ALLOW PDB NAME CHECK!
def get_file_names(dirpath):
    if os.path.exists('input'):
        folder_content = [name_of_the_file for name_of_the_file in os.listdir(dirpath) if name_of_the_file.endswith(".pdb") and os.path.isfile(os.path.join(dirpath, name_of_the_file))]
        if folder_content == []:
            logging.error("No pdb files in the input folder. Exiting.")
            (exit)
        else:
            folder_content.sort(key=lambda name_of_the_file: os.path.getmtime(os.path.join(dirpath, name_of_the_file)))
            output = open("filelist.txt", "w")
            for file_number in range (len(folder_content)):
                output.write("input/"+(folder_content[file_number])+"\n")
            output.close()
        if len(folder_content) == 1:
          logging.info("One pdb file found, check filelist.txt")
        elif len(folder_content) > 1:
            logging.info(str(len(folder_content))+" pdb files found, check filelist.txt.")
        get_file_names.number_of_pdbs=len(folder_content) 
    else:
        logging.error("input folder not found.")
        os.mkdir("input")
        logging.info("input folder created. Please fill it with PDBs and restart the script. Exiting.")
        (exit)

PARSING CLASSES

In [ ]:
# # This list allows to retrieve atoms based on the name or the index
# If standard, dictionary type indexing is used, only exact matches are
# returned. Alternatively, partial matching can be achieved by setting
# a second 'True' argument. 
class Residue(list):
    def __getitem__(self,tag): 
        if type(tag) == int:
            # Call the parent class __getitem__
            return list.__getitem__(self,tag)
        if type(tag) == str:
            for i in self:
                if i[0] == tag:
                    return i
            else:
                return 
        if tag[1]:
            return [i for i in self if tag[0] in i[0]] # Return partial matches
        else:
            return [i for i in self if i[0] == tag[0]] # Return exact matches only

class Chain:
    # Attributes defining a chain
    # When copying a chain, or slicing, the attributes in this list have to
    # be handled accordingly.
    _attributes = ("residues","sequence","seq")

    def __init__(self,residuelist=[],name=None):
        self.residues   = residuelist
        self._atoms     = [atom[:3] for residue in residuelist for atom in residue]
        self.sequence   = [residue[0][1] for residue in residuelist]
        # *NOTE*: Check for unknown residues and remove them if requested
        #         before proceeding.
        self.seq        = "".join([AA321.get(i,"X") for i in self.sequence])
            
        # Unknown residues
        self.unknowns   = "X" in self.seq

        # Determine the type of chain
        self._type      = ""
        self.type()

        # Determine number of atoms
        self.natoms     = len(self._atoms) 

        # Chain identifier; try to read from residue definition if no name is given
        self.id         = name or residuelist and residuelist[0][0][3] or ""

        # Container for coarse grained beads
        self._cg        = None  #added

    def __len__(self):
        # Return the number of residues
        # DNA/RNA contain non-CAP d/r to indicate type. We remove those first.
        return len(''.join(i for i in self.seq if i.isupper()))

    def __add__(self,other):
        newchain = Chain(name=self.id+"+"+other.id)
        # Combine the chain items that can be simply added
        for attr in self._attributes:
            setattr(newchain, attr, getattr(self,attr) + getattr(other,attr))
        # Set chain items, shifting the residue numbers
        newchain.natoms     = len(newchain.atoms())
        # Return the merged chain
        return newchain

    def __eq__(self,other):
        return (self.seq        == other.seq  )#  and 
                #self.breaks     == other.breaks )

    def __ne__(self, other):
        return not self.__eq__(other)

    # Extract a residue by number or the list of residues of a given type
    # This facilitates selecting residues for links, like chain["CYS"]
    def __getitem__(self,other):
        if type(other) == str:
            if not other in self.sequence:
                return []
            return [i for i in self.residues if i[0][1] == other]
        if isinstance(other, slice):
            newchain = Chain(name=self.id)
            for attr in self._attributes:           
                setattr(newchain, attr, getattr(self,attr)[other])
            newchain.natoms     = len(newchain.atoms())
            newchain.type()
            # Return the chain slice
            return newchain


    def _contains(self,atomlist,atom):
        atnm,resn,resi,chn = atom
        
        # If the chain does not match, bail out
        if chn != self.id:
            return False

        # Check if the whole tuple is in
        if atnm and resn and resi:
            return (atnm,resn,resi) in self.atoms()

        # Fetch atoms with matching residue id
        match = (not resi) and atomlist or [j for j in atomlist if j[2] == resi]
        if not match:
            return False

        # Select atoms with matching residue name
        match = (not resn) and match or [j for j in match if j[1] == resn]
        if not match:
            return False

        # Check whether the atom is given and listed
        if not atnm or [j for j in match if j[0] == atnm]:
            return True

        # It just is not in the list!
        return False

    def __contains__(self,other):
        return self._contains(self.atoms(),other) or self._contains(self.cg(),other)

    def __hash__(self):
        return id(self)

    def atoms(self):
        if not self._atoms:
            self._atoms = [atom[:3] for residue in self.residues for atom in residue]
        return self._atoms

    # Split a chain based on residue types; each subchain can have only one type
    def split(self):
        chains = []
        chainStart = 0
        for i in range(len(self.sequence)-1):
            if residueTypes.get(self.sequence[i],"Unknown") != residueTypes.get(self.sequence[i+1],"Unknown"):
                chains.append(self[chainStart:i+1])
                chainStart = i+1
        if chains:
            logging.debug('Splitting chain %s in %s chains'%(self.id,len(chains)+1))
        return chains + [self[chainStart:]] 

    def getname(self,basename=None):
        name = []
        if basename:                      name.append(basename)
        if self.type() and not basename:  name.append(self.type())
        if type(self.id) == int:
            name.append(chr(64+self.id))
        elif self.id.strip():               
            name.append(str(self.id))
        return "_".join(name)

    def type(self,other=None):
        if other:
            self._type = other
        elif not self._type and len(self):
            # Determine the type of chain
            self._type     = set([residueTypes.get(i,"Unknown") for i in set(self.sequence)])
            self._type     = list(self._type)[0]
        return self._type

PARAMETERS AND DICTIONARIES

In [ ]:
dnares3 = " DA DC DG DT" 
dnares1 = " dA dC dG dT"
rnares3 = "  A  C  G  U"
rnares1 = " rA rC rG rU" # 

# Amino acid nucleic acid codes:                                                                                 
# The naming (AA and '3') is not strictly correct when adding DNA/RNA, but we keep it like this for consistincy.
AA3     = spl("TRP TYR PHE HIS HIH HIP HID HIE HSE HSD HSP ARG LYS CYS ASP GLU ILE LEU MET ASN PRO HYP GLN SER THR VAL ALA GLY"+dnares3+rnares3)
AA1     = spl("  W   Y   F   H   H   H   H   H   H   H   H   R   K   C   D   E   I   L   M   N   P   O   Q   S   T   V   A   G"+dnares1+rnares1)


# Dictionaries for conversion from one letter code to three letter code v.v.                         
AA123, AA321 = hash(AA1,AA3),hash(AA3,AA1)                                                           


# Residue classes:
protein = AA3[:-8]   # remove eight to get rid of DNA/RNA here.
water   = spl("HOH SOL TIP WAT")
lipids  = spl("DPP DHP DLP DMP DSP POP DOP DAP DUP DPP DHP DLP DMP DSP PPC DSM DSD DSS")
nucleic = spl("DAD DCY DGU DTH ADE CYT GUA THY URA DA DC DG DT A C G U")
ions = spl("CL NA Cl Na Cl- Na+")

# Backbone & Masses
bbHA        = "N CA C O H HN H1 H2 H3 O1 O2 HA HA1 HA2 OC1 OC2"
mass = {'H': 1,'C': 12,'N': 14,'O': 16,'S': 32,'P': 31,'M': 0} 

residueTypes = dict(
    [(i,"Protein") for i in protein ]+ 
    [(i,"Water")   for i in water   ]+      #deprecated
    [(i,"Lipid")   for i in lipids  ]+
    [(i,"Nucleic") for i in nucleic ]+
    [(i,"Ions") for i in ions ]             #deprecated
    )

Q -VALUES

In [ ]:
qval=np.arange(0.000,0.700,0.001)   # U.M.: ANG-1 #from,to,stride)

MARTINI MAPPING - CLASSES AND FXs

In [ ]:
# Split each argument in a list                                               
def nsplit(*x):                                                               
    return [i.split() for i in x]                                             

class martini:
    # Class for mapping an atomistic residue list to a martini coarsegrained one

    # Standard mapping groups
    dna_bb = "P OP1 OP2 O5' O3'","C5' O4' C4'","C3' C2' C1'"
    rna_bb = "P OP1 OP2 O5' O3'","C5' O4' C4'","C3' C2' O2' C1'"

    # This is the mapping dictionary including Hydrogen atoms
    mapping = {
        "ALA":  nsplit(bbHA + " CB HB1 HB2 HB3"),
        "CYS":  nsplit(bbHA,"CB SG HB1 HB2 HG HG1"),
        "ASP":  nsplit(bbHA,"CB CG OD1 OD2 HB1 HB2"),
        "GLU":  nsplit(bbHA,"CB CG CD OE1 OE2 HB1 HB2 HG1 HG2"),
        "PHE":  nsplit(bbHA,"CB CG CD1 HD1 HB1 HB2","CD2 HD2 CE2 HE2","CE1 HE1 CZ HZ"),
        "GLY":  nsplit(bbHA),
        "HIS":  nsplit(bbHA,"CB  HB1 HB2 CG","CD2 HD2 NE2 HE2","ND1 HD1 CE1 HE1"),
        "HIP":  nsplit(bbHA,"CB  HB1 HB2 CG","CD2 HD2 NE2 HE2","ND1 HD1 CE1 HE1"),
        "HID":  nsplit(bbHA,"CB  HB1 HB2 CG","CD2 HD2 NE2 HE2","ND1 HD1 CE1 HE1"),
        "HIE":  nsplit(bbHA,"CB  HB1 HB2 CG","CD2 HD2 NE2 HE2","ND1 HD1 CE1 HE1"),
        "HSP":  nsplit(bbHA,"CB  HB1 HB2 CG","CD2 HD2 NE2 HE2","ND1 HD1 CE1 HE1"),
        "HSD":  nsplit(bbHA,"CB  HB1 HB2 CG","CD2 HD2 NE2 HE2","ND1 HD1 CE1 HE1"),
        "HSE":  nsplit(bbHA,"CB  HB1 HB2 CG","CD2 HD2 NE2 HE2","ND1 HD1 CE1 HE1"),
        "ILE":  nsplit(bbHA,"CB CG1 CG2 CD CD1 HB HG21 HG22 HG23 HG11 HG12 HG13 HD1 HD2 HD3"),
        "LYS":  nsplit(bbHA,"CB CG CD HB1 HB2 HG1 HG2 HD1 HD2","CE HE1 HE2 NZ HZ1 HZ2 HZ3"),
        "LEU":  nsplit(bbHA,"CB CG CD1 CD2 HB1 HB2 HG HD11 HD12 HD13 HD21 HD22 HD23"),
        "MET":  nsplit(bbHA,"CB CG SD CE HB1 HB2 HG1 HG2 HE1 HE2 HE3"),
        "ASN":  nsplit(bbHA,"CB HB1 HB2 CG ND1 ND2 OD1 OD2 HD11 HD12 HD21 HD22"),
        "PRO":  nsplit(bbHA,"CB CG CD HB1 HB2 HG1 HG2 HD1 HD2"),
        "HYP":  nsplit(bbHA,"CB CG CD OD HB1 HB2 HG1 HG2 HD1 HD2 HD"),
        "GLN":  nsplit(bbHA,"CB CG HB1 HB2 HG1 HG2 CD OE1 OE2 NE1 NE2 HE11 HE12 HE21 HE22"),        
        "ARG":  nsplit(bbHA,"CB CG CD HB1 HB2 HG1 HG2 HD1 HD2","NE HE CZ NH1 NH2 HH11 HH12 HH21 HH22"),    
        "SER":  nsplit(bbHA,"CB HB1 HB2 OG HG HG1"),
        "THR":  nsplit(bbHA,"CB HB OG1 HG1 CG2 HG21 HG22 HG23 OG2 HG2 CG1 HG11 HG12 HG13"),
        "VAL":  nsplit(bbHA,"CB CG1 CG2 HB HG21 HG22 HG23 HG11 HG12 HG13"),
        "TRP":  nsplit(bbHA,"CB CG CD2 HB1 HB2","CD1 HD1 NE1 HE1 CE2","CE3 HE3 CZ3 HZ3","CZ2 HZ2 CH2 HH2"),
        "TYR":  nsplit(bbHA,"CB HB1 HB2 CG CD1 HD1","CD2 HD2 CE2 HE2","CE1 HE1 CZ OH HH"),
        "DA": nsplit("P OP1 OP2 O5' O3' O1P O2P",
                          "C5' O4' C4' H5'1 H5'2 H4'",
                          "C3' C2' C1' H1' H3' H2'1 H2'2",
                          "N9 C4",
                          "C2 N3 H2",
                          "C6 N6 N1 H61 H62",
                          "C8 N7 C5 H8"),
        "DG": nsplit("P OP1 OP2 O5' O3' O1P O2P",
                          "C5' O4' C4' H5'1 H5'2 H4'",
                          "C3' C2' C1' H1' H3' H2'1 H2'2",
                          "N9 C4",
                          "C2 N2 N3 H21 H22",
                          "C6 O6 N1 H1",
                          "C8 N7 C5 H8"),
        "DC": nsplit("P OP1 OP2 O5' O3' O1P O2P",
                          "C5' O4' C4' H5'1 H5'2 H4'",
                          "C3' C2' C1' H1' H3' H2'1 H2'2",
                          "N1 C6 H6",
                          "N3 C2 O2",
                          "C5 C4 N4 H5 H41 H42"),
        "DT": nsplit("P OP1 OP2 O5' O3' O1P O2P",
                          "C5' O4' C4' H5'1 H5'2 H4'",
                          "C3' C2' C1' H1' H3' H2'1 H2'2",
                          "N1 C6 H6",
                          "N3 C2 O2 H3",
                          "C5 C4 O4 C7 C5M H71 H72 H73 H5"),
        "A":  nsplit("P OP1 OP2 O5' O3' O1P O2P",
                          "C5' O4' C4' H5'1 H5'2 H4'",
                          "C3' C2' O2' C1' H1' H3' H2' H2'1 HO'2",
                          "N9 C4",
                          "C2 N3 H2",
                          "C6 N6 N1 H61 H62",
                          "C8 N7 C5 H8"),
        "G":  nsplit("P OP1 OP2 O5' O3' O1P O2P",
                          "C5' O4' C4' H5'1 H5'2 H4'",
                          "C3' C2' O2' C1' H1' H3' H2' H2'1 HO'2",
                          "N9 C4",
                          "C2 N2 N3 H21 H22",
                          "C6 O6 N1 H1",
                          "C8 N7 C5 H8"),
        "C":  nsplit("P OP1 OP2 O5' O3' O1P O2P",
                          "C5' O4' C4' H5'1 H5'2 H4'",
                          "C3' C2' O2' C1' H1' H3' H2' H2'1 HO'2",
                          "N1 C6 H6",
                          "N3 C2 O2",
                          "C5 C4 N4 H5 H41 H42"),
        "U":  nsplit("P OP1 OP2 O5' O3' O1P O2P",
                          "C5' O4' C4' H5'1 H5'2 H4'",
                          "C3' C2' O2' C1' H1' H3' H2' H2'1 HO'2",
                          "N1 C6 H6",
                          "N3 C2 O2 H3",
                          "C5 C4 O4 C7 C5M H71 H72 H73 H5"),
        } 


    # Generic names for side chain beads
    residue_bead_names = spl("BB SC1 SC2 SC3 SC4")
    # Generic names for DNA/RNA beads
    residue_bead_names_dna = spl("BB1 BB2 BB3 SC1 SC2 SC3 SC4")
    residue_bead_names_rna = spl("BB1 BB2 BB3 SC1 SC2 SC3 SC4")

    # This dictionary contains the bead names for all residues,
    # following the order in 'mapping'
    # Add default bead names for all amino acids
    names = {}
    names.update([(i,("BB","SC1","SC2","SC3","SC4")) for i in AA3])
    # Add the default bead names for all DNA nucleic acids
    names.update([(i,("BB1","BB2","BB3","SC1","SC2","SC3","SC4")) for i in nucleic])

FORM FACTOR CLASS

In [ ]:
class AtomicFormFactor:

    # ATOMIC FORM FACTOR
    # REFERENCES
    ##### theory/formula etc.:
    # Computational and Structural Biotechnology Journal,  Vol 8, Issue 11, e201308006, http://dx.doi.org/10.5936/csbj.201308006. Reconstruction of SAXS Profiles from Protein Structures
    # Fraser et al. J. Appl. Cryst. (1978). 11, 693-694 
    ##### a/b/c parameters (ATT! there are two options for H)
    # International Tables for Crystalloraphy (2006). Vol. C, Chapter 6.1, pp. 554-595 
    ##### Volume v
    # Fraser et al. J. Appl. Cryst. (1978). 11, 693-694 
    # J. Appl. Cryst. (1995), 28, 768-733. CRYSOL 
    ##### Mean electron density of bulk water [rho]
    # Computational and Structural Biotechnology Journal,  Vol 8, Issue 11, e201308006, http://dx.doi.org/10.5936/csbj.201308006. Reconstruction of SAXS Profiles from Protein Structures
    # see also: https://books.google.it/books?id=gTAWBAAAQBAJ&pg=PA593&lpg=PA593&dq=water+mean+electron+density+0.334&source=bl&ots=x_E9rMwEcY&sig=bpsvc3oh7-ZWlzGleW5NaI9lWh0&hl=it&sa=X&ved=0ahUKEwjdx6WRhr_aAhWkw6YKHQzUAVoQ6AEIMDAB#v=onepage&q=water%20mean%20electron%20density%200.334&f=false
    # rho=0.334 e-/Ang3 for pure water; rho=0.40  e-/Ang3 for salt solution with higher density; crystallographic programs often use a compromise value around 0.375 e-/Ang3
    # Here we use 0.334; consider the possibility to use 0.375

    param_a = []; param_b = []
    for i in range(5):
        param_a.append({ })
        param_b.append({ })
    param_c = { }
    param_v = { }

    rho=0.334

    # H parameters to reproduce spherical bonded H-atoms scattering factors from Stewart, Davidson and Simpson (1965)
    # Table 6.1.1.2 and 6.1.1.3 of International Tables for Crystalloraphy (2006). Vol. C, Chapter 6.1, pp. 554-595
    param_a[0]['H'] = 0.493002; param_b[0]['H'] = 10.5109; param_c['H'] = 0.003038;
    param_a[1]['H'] = 0.322912; param_b[1]['H'] = 26.1257; param_v['H'] = 5.15;
    param_a[2]['H'] = 0.140191; param_b[2]['H'] = 3.14236;
    param_a[3]['H'] = 0.040810; param_b[3]['H'] = 57.7997;
    
    # H parameters to reproduce H scattering factor calculated from the analytical solution to the Schr equation
    # It does not consider correction to scattering factor for bonded H (for other atoms this correction is less relevant)
    # Table 6.1.1.1 and 6.1.1.3 of International Tables for Crystalloraphy (2006). Vol. C, Chapter 6.1, pp. 554-595
    # RESULTS ARE ALMOST IDENTICAL USING THIS OR CORRECTED SCATTERING FACTORS
    #param_a[0]['H'] = 0.489918; param_b[0]['H'] = 20.6593; param_c['H'] = 0.001305;
    #param_a[1]['H'] = 0.262003; param_b[1]['H'] = 7.74039; param_v['H'] = 5.15;
    #param_a[2]['H'] = 0.196767; param_b[2]['H'] = 49.5519;
    #param_a[3]['H'] = 0.049879; param_b[3]['H'] = 2.20159

    param_a[0]['C'] = 2.31000; param_b[0]['C'] = 20.8439; param_c['C'] = 0.215600;
    param_a[1]['C'] = 1.02000; param_b[1]['C'] = 10.2075; param_v['C'] = 16.44;
    param_a[2]['C'] = 1.58860; param_b[2]['C'] = 0.56870;
    param_a[3]['C'] = 0.86500; param_b[3]['C'] = 51.6512;

    param_a[0]['N'] = 12.2126; param_b[0]['N'] = 0.00570; param_c['N'] = -11.529;
    param_a[1]['N'] = 3.13220; param_b[1]['N'] = 9.89330; param_v['N'] = 2.49;
    param_a[2]['N'] = 2.01250; param_b[2]['N'] = 28.9975;
    param_a[3]['N'] = 1.16630; param_b[3]['N'] = 0.58260;

    param_a[0]['O'] = 3.04850; param_b[0]['O'] = 13.2771; param_c['O'] = 0.250800 ;
    param_a[1]['O'] = 2.28680; param_b[1]['O'] = 5.70110; param_v['O'] = 9.13;
    param_a[2]['O'] = 1.54630; param_b[2]['O'] = 0.32390;
    param_a[3]['O'] = 0.86700; param_b[3]['O'] = 32.9089;

    param_a[0]['P'] = 6.43450; param_b[0]['P'] = 1.90670; param_c['P'] = 1.11490;
    param_a[1]['P'] = 4.17910; param_b[1]['P'] = 27.1570; param_v['P'] = 5.73;
    param_a[2]['P'] = 1.78000; param_b[2]['P'] = 0.52600;
    param_a[3]['P'] = 1.49080; param_b[3]['P'] = 68.1645;

    param_a[0]['S'] = 6.90530; param_b[0]['S'] = 1.46790; param_c['S'] = 0.866900;
    param_a[1]['S'] = 5.20340; param_b[1]['S'] = 22.2151; param_v['S'] = 19.86;
    param_a[2]['S'] = 1.43790; param_b[2]['S'] = 0.25360;
    param_a[3]['S'] = 1.58630; param_b[3]['S'] = 56.1720;

FORM FACTOR FXs

In [ ]:
def distance2(a,b):
    return (a[0]-b[0])**2+(a[1]-b[1])**2+(a[2]-b[2])**2

def FormFactor(atomn,qlist):
    f = np.ones(len(qlist))
    f = np.multiply(f,AtomicFormFactor.param_c[atomn])
    s = np.power(qlist/(4.*np.pi),2.)
    volt = np.power(AtomicFormFactor.param_v[atomn],2./3.)/(4.*np.pi)
    for k in range(4):
        f += np.multiply(AtomicFormFactor.param_a[k][atomn],np.exp(np.multiply(-AtomicFormFactor.param_b[k][atomn],s)))
    f -= np.multiply(AtomicFormFactor.rho*AtomicFormFactor.param_v[atomn],np.exp(np.multiply(-volt,np.power(qlist,2.))))
    return f

def beadff(b,qval,atomicff):
    beadffv=0;
    for a1 in b:
        for a2 in b:
            if a2[5] >= a1[5]: #aid
                if a2[5] == a1[5]:
                    beadffv +=  np.multiply(atomicff[a1[1]],atomicff[a2[1]]) #t
                else:
                    qr=np.multiply(qval,np.sqrt(distance2(a1[2],a2[2]))) #coord
                    qr_over_pi=np.divide(qr,np.pi) # needed to use np.sinc
                    beadffv +=  2*np.multiply(np.multiply(atomicff[a1[1]],atomicff[a2[1]]),np.sinc(qr_over_pi))
    beadffsq =np.sqrt(beadffv)
    return beadffsq

def beadff0(b,atomicff):
    beadffv=0;
    for a1 in b:
        beadffv +=  atomicff[a1[1]][0]
    return beadffv

# SIXTH-ORDER POLINOMIAL FIT FUNCTION
def func6(x,a0,a1,a2,a3,a4,a5,a6):
    return a0+a1*x+a2*x**2+a3*x**3+a4*x**4+a5*x**5+a6*x**6
   
def fit_func6(bff,bff0,qval):
    if np.abs(bff[0]) - np.abs(bff0) > np.finfo(np.float32).eps:
        logging.error("Something is wrong in the form factor calculations for q=0. Exiting...")
    else:
        lowb=np.multiply(np.ones(7),-np.inf);     
        upb=np.multiply(np.ones(7),np.inf);  
        lowb[0]=bff0-np.finfo(np.float32).eps;
        upb[0]=bff0+np.finfo(np.float32).eps;
        if bff0 > 0:
            fit_func6.checker=False
            popt, pcov = curve_fit(func6, qval, bff, bounds=(lowb,upb))
        else:
            fit_func6.checker=True
            ind=np.diff(bff).argmax() # find high q inflection point and fit only of the part of the curve after this point
            popt, pcov = curve_fit(func6, qval[ind+1:], bff[ind+1:], bounds=(lowb,upb))
    return popt

MAPS ASSEMBLING FXs

In [ ]:
def listatoms(residue):
    a=[]
    #atomtype definition
    for i in residue:
        if i[0][0] in mass.keys():
            attype=i[0][0]
        else:
            if i[0][1] in mass.keys():
                attype=i[0][1]
            else:
                logging.warning("Atom %s not recognized" %i[0])

        a.append((i[0],attype,i[4:7],i[7],mass.get(attype,0)))
    return a

def mapMono(residue):
    a=listatoms(residue)
    monoMap = [[(m,t,coord,a.index((atom,t,coord,aid,m)),atom,aid) for atom,t,coord,aid,m in a]]
    return monoMap
      
def mapDual(residue):
    a=listatoms(residue)
    dual_bb = [(m,t,coord,a.index((atom,t,coord,aid,m)),atom,aid) for atom,t,coord,aid,m in a if atom in bbHA]
    dual_sc = [(m,t,coord,a.index((atom,t,coord,aid,m)),atom,aid) for atom,t,coord,aid,m in a if not atom in bbHA]
    dualMap = [dual_bb, dual_sc]
    return dualMap
                 
def mapMartini(residue):           
    a=listatoms(residue)  
    p = martini.mapping[residue[0][1]]
    martiniMap = [[(m,t,coord,a.index((atom,t,coord,aid,m)),atom,aid) for atom,t,coord,aid,m in a if atom in i] for i in p]                                                    

    flat_p = [item for sublist in p for item in sublist]
    martiniNOTinb = [(atom,aid) for atom,t,coord,aid,m in a if atom not in flat_p]
    if len(martiniNOTinb):
       logging.warning("MISSING: ",fname,ci.id,resname,martiniNOTinb)
    return martiniMap


def AVER(b):
    mwx,ids,atom,aids = list(zip(*[((m*x,m*y,m*z),i,at,aid) for m,t,(x,y,z),i,at,aid in b]))
    tm  = sum(next(zip(*b)))
    return [sum(i)/tm for i in zip(*mwx)],ids,atom,aids #CoM 

FILES MANAGING FXs

In [ ]:
def backupper(filename):
    if os.path.isfile(filename):
        os.rename(filename, filename+".bkp_"+str(datetime.today().strftime('%Y%m%d%H%M%S')))
        logging.info(filename+' found, backup done.' )

def folder_backupper(foldername):
    full_name=foldername+"_"+(str(get_file_names.number_of_pdbs))
    if os.path.exists(full_name):
        os.rename(full_name, full_name+"-bkp_"+str(datetime.today().strftime('%Y%m%d%H%M%S')))
        logging.info('Folder '+full_name+' found, backup done.' )
    os.mkdir(full_name)

def file_checker(filename1,filename2):
    if os.path.isfile(filename1) and os.path.isfile(filename2):
        logging.info(filename1+' and '+filename2+' have been successfully written.')
    else:
        logging.warning('Something went wrong while writing '+filename1+' and '+filename2+'.')

def file_checker_single(filename):
    if os.path.isfile(filename):
        logging.info(filename+' has been successfully written.')
    else:
        logging.warning('Something went wrong while writing '+filename+'.')

PRINTER FXs

In [ ]:
def print_beadff00(fname,bff0):
    with open(fname, 'w') as f:
        f.write(str(bff0))

def print_beadff(bff,fname):
    outFF = open(fname,"a")
    np.savetxt(outFF,bff,newline=' ',fmt="%.10f") #10 DECIMALS AFTER POINT
    outFF.write("\n")
    outFF.close()

def plumed_main_printer(filename,include,aver,ff0,bead_name):
    param_counter=0; negative_fitting_counter=0
    plumedmain = open(filename,"w")
    plumedmain.write("MOLINFO STRUCTURE=template_AA.pdb\n\n# BEADS DEFINITION\nINCLUDE FILE="+include+"\n\n")
    plumedmain.write("# SAXS\nSAXS ...\n\n\tLABEL=saxsdata\n\tATOMS="+bead_name+"\n\n\t#SAXS CALCULATION CAN BE ACCELERATED WITH GPU, PLUMED COMPILED WITH ARRAYFIRE IS REQUIRED\n\t#GPU\n\t#DEVICEID\n\n\t# You can use SCALEINT keyword to set appropriate scaling factor.\n\t# SCALEINT is expected to correspond to the intensity in q=0\n\t# SCALEINT=\n\n")

    for resnm in list(aver.keys()):
        coeffF6=fit_func6(aver[resnm],ff0[resnm],qval)
        plumedmain.write("PARAMETERS%d=%f,%f,%f,%f,%f,%f,%f # %s \n" %(param_counter+1,*coeffF6,resnm)) # %d decimal integer - %s string 
        param_counter+=1
        if fit_func6.checker==True:
            negative_fitting_counter+=1
    if negative_fitting_counter == 1:
            logging.warning(bead_name+" mapping: "+str(negative_fitting_counter)+" bead Form Factor is negative at q=0, the estimated PLUMED parameters are the result of a curve fitting.")
    if negative_fitting_counter > 1:
            logging.warning(bead_name+" mapping: "+str(negative_fitting_counter)+" bead Form Factors are negative at q=0, the estimated PLUMED parameters are the result of curve fitting.")
    plumedmain.write("\nQVALUE1=0.00000001 #EXPINT1=\nQVALUE2=0.01 #EXPINT2=\nQVALUE3=0.02 #EXPINT3=\nQVALUE4=0.03 #EXPINT4=\nQVALUE5=0.04 #EXPINT5=\nQVALUE6=0.05 #EXPINT6=\nQVALUE7=0.06 #EXPINT7=\nQVALUE8=0.07 #EXPINT8=\nQVALUE9=0.08 #EXPINT9=\nQVALUE10=0.09 #EXPINT10=\nQVALUE11=0.10 #EXPINT11=\nQVALUE12=0.11 #EXPINT12=\nQVALUE13=0.12 #EXPINT13=\nQVALUE14=0.13 #EXPINT14=\nQVALUE15=0.14 #EXPINT15=\nQVALUE16=0.15 #EXPINT16=\nQVALUE17=0.16 #EXPINT17=\nQVALUE18=0.17 #EXPINT18=\nQVALUE19=0.18 #EXPINT19=\nQVALUE20=0.19 #EXPINT20=\nQVALUE21=0.20 #EXPINT21=\nQVALUE22=0.21 #EXPINT22=\nQVALUE23=0.22 #EXPINT23=\nQVALUE24=0.23 #EXPINT24=\nQVALUE25=0.24 #EXPINT25=\nQVALUE26=0.25 #EXPINT26=\nQVALUE27=0.26 #EXPINT27=\nQVALUE28=0.27 #EXPINT28=\nQVALUE29=0.28 #EXPINT29=\nQVALUE30=0.29 #EXPINT30=\nQVALUE31=0.30 #EXPINT31=\n")
    plumedmain.write("\n\t# METAINFERENCE\n\t# Uncomment the following keywords and adjust parameters to activate METAINFERENCE\n\t# DOSCORE NOENSEMBLE SIGMA_MEAN0=0\n\t# REGRES_ZERO=500\n\t# SIGMA0=5 SIGMA_MIN=0.001 SIGMA_MAX=5.00\n\t# NOISETYPE=MGAUSS\n\n... SAXS\n")
    plumedmain.write("\n# METAINFERENCE\n# Uncomment the following keyword to activate METAINF\n# saxsbias: BIASVALUE ARG=(saxsdata\.score) STRIDE=10\n# STATISTICS\n#statcg: STATS ARG=(saxsdata\.q-.*) PARARG=(saxsdata\.exp_.*)\n")
    plumedmain.write("\n# PRINT\n\n# Uncomment the following line to print METAINFERENCE output\n# PRINT ARG=(saxsdata\.score),(saxsdata\.biasDer),(saxsdata\.weight),(saxsdata\.scale),(saxsdata\.offset),(saxsdata\.acceptSigma),(saxsdata\.sigma.*) STRIDE=500 FILE=BAYES.SAXS\n\n# change stride if you are using METAINFERENCE\nPRINT ARG=(saxsdata\.q-.*) STRIDE=1 FILE=SAXSINT\n#PRINT ARG=statcg.corr STRIDE=1 FILE=ST.SAXSCG\n")
    plumedmain.close()

EXTERNAL READER FXs

In [ ]:
def format_beads_number(number,number_of_beads):
    number_of_digits=(len(str(number_of_beads)))
    return (str(number).zfill(number_of_digits))

def string_finder(string, key):
    index = 0
    string_finder.checker=False
    if key in string:
        c = key[0]
        for ch in string:
            if ch == c:
                if string[index:index+len(key)] == key:
                    return index
            index += 1
    else:
        logging.error("The file name does not contain \""+key+"\"")
        string_finder.checker=True
        exit()
    return -1

def ext_reader(folder,skip):
    if any(fname.endswith('.ff') for fname in os.listdir(folder)):
        logging.info("FF directory found")
        pth = Path(folder).glob('*.ff')
        pth = [Path(p) for p in sorted(pth)]
        for path in pth:
            pos=string_finder(path.stem,"chain")
            if string_finder.checker == True:
                break
            yield path.stem[pos:], np.genfromtxt(str(path),skip_footer=skip)
    else:
        logging.error("Files not found in: "+folder+".")
        exit()

def bead_selector(name):
    selector_type=(name)
    if selector_type == "mono":
        bead_selector.plumed_main=("plumed_main_1B_ext.dat")
        bead_selector.plumed_include=("plumed_1B.dat")
        bead_selector.bead_name=("mono")
    elif selector_type == "dual":
        bead_selector.plumed_main=("plumed_main_2B_ext.dat")
        bead_selector.plumed_include=("plumed_2B.dat")
        bead_selector.bead_name=("dual")
    elif selector_type == "martini":
        bead_selector.plumed_main=("plumed_main_martini_ext.dat")
        bead_selector.plumed_include=("plumed_martini.dat")
        bead_selector.bead_name=("martini")
    elif selector_type != "mono" and selector_type != "dual" and selector_type != "martini":
        logging.error("Bead type not recognized.")
        exit()

OTHER FXs

In [ ]:
def clean_whitespaces(string):
    return string.replace(" ","")

MAIN - READER

In [ ]:
get_file_names("input")

# PDB file list generated by get_file_names
filelist=('filelist.txt')

# Reformatting of lines in structure file
pdbAtomLine = "ATOM  %5d %4s%4s %1s%4d%1s   %8.3f%8.3f%8.3f%6.2f%6.2f\n"

with open(filelist) as fo:
    files = fo.read().splitlines()
    nf=0
    ListOfPDB = []
    for f in files:
        if not f.startswith("#") and f.strip():
            if f[-4:] !=".pdb":
                logging.warning("In file list %s is not a pdb." %f)
                continue
                                
            fname=f[:-4]
            nf+=1

            model=1
            inStream = streamTag(f)
            for title,atoms in pdbFrameIterator(inStream):
  
                chains = [ Chain([i for i in residues(chain)]) for chain in pdbChains(atoms) ]

                if model > 1: logging.warning("More then one model found in your pdb file %s.\nThis could cause error in the script, pay attention." %f )
                model +=1

                if len(chains) != len(set([i.id for i in chains])):
                # Ending down here means that non-consecutive blocks of atoms in the 
                # PDB file have the same chain ID.
                    logging.warning("Several chains have identical chain identifiers in the PDB file.")
   
                # Check if chains are of mixed type. If so, split them.
                demixedChains = []
                for chain in chains:
                    demixedChains.extend(chain.split())
                chains = demixedChains

                n = 1
                logging.info("Found %d chains:"%len(chains))
                for chain in chains:
                    logging.info("  %2d:   %s (%s), %d atoms in %d residues."%(n,chain.id,chain._type,chain.natoms,len(chain)))
                    n += 1
    
                # Check all chains
                keep = []
                for chain in chains:
                    if chain.type() == "Water":
                        logging.info("Removing %d water molecules (chain %s)."%(len(chain),chain.id))
                    elif chain.type() in ("Protein","Nucleic"):
                        keep.append(chain)
                    else:
                        logging.info("Removing HETATM chain %s consisting of %d residues."%(chain.id,len(chain)))
                chains = keep

                
                # Get the total length of the sequence
                seqlength = sum([len(chain) for chain in chains])
                logging.info('Total size of the system: %s residues.'%seqlength)
    
                if nf == 1:
                    order = []
                    order.extend([j for j in range(len(chains)) if not j in order])
                else:
                    # check if all files match in sequence reference file 0
                    if len(chains) == len(order):
                        for i in order:
                            if chains[i].sequence != ListOfPDB[0][i].sequence:
                                logging.warning("Sequence of chain %d in file 0 and %s differ." %(i,f))
                    else:
                        logging.warning("Different number of chains in file 0 and %s" %f)

                ListOfPDB.append(chains)

SWITCHES

In [ ]:
MONO_CG_switch = True
DUAL_CG_switch = True
MARTINI_CG_switch = True

MONO_FF_switch = True
DUAL_FF_switch = True
MARTINI_FF_switch = True

debug_AVER=True
debug_FF=False
debug_FF0=True

EXTERNAL_FXs = False

MAIN - STARTING

In [ ]:
if MONO_CG_switch == True:
        backupper("plumed_main_1B.dat")
        backupper("template_1B.pdb")
if MONO_FF_switch == True:
        folder_backupper('1B')
        if debug_FF0 == True:
                folder_backupper('1B_ff0')

if DUAL_CG_switch == True:    
        backupper("plumed_main_2B.dat")
        backupper("template_2B.pdb")
if DUAL_FF_switch == True:    
        folder_backupper('2B')
        if debug_FF0 == True:
                folder_backupper('2B_ff0')        

if MARTINI_CG_switch == True:
        backupper("plumed_martini.dat")
        backupper("template_martini.pdb")
if MARTINI_FF_switch == True:
        folder_backupper('martini')
        if debug_FF0 == True:
                folder_backupper('martini_ff0')


if debug_AVER==True:
    logging.info("Debug mode activated: averaged FF will be printed.")

if debug_FF==True:
    logging.warning("Debug mode activated: all the FF will be printed.")
 

#COPYING FRAME0 to TEMPLATE_AA
if os.path.isfile(files[0]):   
        os.system('cp '+files[0]+" template_AA.pdb")
else:
        logging.warning('Failed to write template_AA.pdb.')

MAIN - COARSE GRAINING

In [ ]:
mono_final_row ="mono: GROUP ATOMS="
dual_final_row ="dual: GROUP ATOMS="
martini_final_row ="martini: GROUP ATOMS="

mono_counter=0; dual_counter=0; martini_counter=0;
map_1b=[];map_2b=[]; map_martini=[];
number_of_beads_mono=0; number_of_beads_dual=0; number_of_beads_martini=martini_counter;

for residue,resname in zip(chains[0].residues,chains[0].sequence):
    resi = residue[0][2]
    insc  = resi >>20
    resi -= insc<<20

#### ONE_BEAD ####
    if MONO_CG_switch == True:
        map_1b=mapMono(residue)
        pos_mono=list(zip(*[AVER(n) for n in map_1b if len(n)>0]))
        mono_masses=[]
        for t in range(0, len(map_1b[0])):
            mono_masses.append(map_1b[0][t][0])
        Out_1B_MAP = open("plumed_1B.dat","a")
        Out_1B_PDB = open("template_1B.pdb","a")
        mono_counter+=1
        Out_1B_MAP.write("bead"+str(mono_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_mono[3][0])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(mono_masses)[1:-1]))+"\n")
        mono_final_row = mono_final_row + "bead" + str(mono_counter)  + ","
        Out_1B_PDB.write(pdbAtomLine%(mono_counter,"RES",resname,"",resi,chr(insc),(pos_mono[0][0][0]),(pos_mono[0][0][1]),(pos_mono[0][0][2]),1,0))


#### TWO_BEADS ####
    if DUAL_CG_switch == True:
        map_2b=mapDual(residue)
        pos_dual=list(zip(*[AVER(n) for n in map_2b if len(n)>0]))
        dual_masses_bb=[]; dual_masses_sc=[];
        for v in range(0, len(map_2b[0])):
            dual_masses_bb.append(map_2b[0][v][0])
        for u in range(0, len(map_2b[1])):
            dual_masses_sc.append(map_2b[1][u][0])
        Out_2B_MAP = open("plumed_2B.dat","a")
        Out_2B_PDB = open("template_2B.pdb","a")
        if (len(pos_dual[3])) == 2: 
            dual_counter+=1
            Out_2B_MAP.write("bead"+str(dual_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_dual[3][0])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(dual_masses_bb)[1:-1]))+"\n")
            dual_final_row = dual_final_row + "bead" + str(dual_counter)  + ","
            Out_2B_PDB.write(pdbAtomLine%(dual_counter,"BB",resname,"",resi,chr(insc),(pos_dual[0][0][0]),(pos_dual[0][0][1]),(pos_dual[0][0][2]),1,0))
            dual_counter+=1
            Out_2B_MAP.write("bead"+str(dual_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_dual[3][1])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(dual_masses_sc)[1:-1]))+"\n")
            dual_final_row = dual_final_row + "bead" + str(dual_counter)  + ","
            Out_2B_PDB.write(pdbAtomLine%(dual_counter,"SC",resname,"",resi,chr(insc),(pos_dual[0][1][0]),(pos_dual[0][1][1]),(pos_dual[0][1][2]),1,0))
        else:               #check for GLY (one bead!)
            dual_counter+=1
            Out_2B_MAP.write("bead"+str(dual_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_dual[3][0])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(dual_masses_bb)[1:-1]))+"\n")
            dual_final_row = dual_final_row + "bead" + str(dual_counter)  + ","
            Out_2B_PDB.write(pdbAtomLine%(dual_counter,"BB",resname,"",resi,chr(insc),(pos_dual[0][0][0]),(pos_dual[0][0][1]),(pos_dual[0][0][2]),1,0))



######## MARTINI_BEADS ####
    if MARTINI_CG_switch==True:
        map_martini=mapMartini(residue)
        pos_martini=list(zip(*[AVER(n) for n in map_martini if len(n)>0]))
        martini_masses_bb=[]; martini_masses_sc1=[]; martini_masses_sc2=[]; martini_masses_sc3=[]; martini_masses_sc4=[];
        Out_martini_MAP = open("plumed_martini.dat","a")
        Out_martini_PDB = open("template_martini.pdb","a")

        if len(map_martini) == 1:
            for d in range(0, len(map_martini[0])):
                martini_masses_bb.append(map_martini[0][d][0])
            martini_counter+=1
            Out_martini_MAP.write("bead"+str(martini_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_martini[3][0])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(martini_masses_bb)[1:-1]))+"\n")
            martini_final_row = martini_final_row + "bead" + str(martini_counter)  + ","
            Out_martini_PDB.write(pdbAtomLine%(martini_counter,"BB",resname,"",resi,chr(insc),(pos_martini[0][0][0]),(pos_martini[0][0][1]),(pos_martini[0][0][2]),1,0))
        elif len(map_martini) == 2:
            for d in range(0, len(map_martini[0])):
                martini_masses_bb.append(map_martini[0][d][0])
            for g in range(0, len(map_martini[1])):
                martini_masses_sc1.append(map_martini[1][g][0])
            martini_counter+=1
            Out_martini_MAP.write("bead"+str(martini_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_martini[3][0])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(martini_masses_bb)[1:-1]))+"\n")
            martini_final_row = martini_final_row + "bead" + str(martini_counter)  + ","
            Out_martini_PDB.write(pdbAtomLine%(martini_counter,"BB",resname,"",resi,chr(insc),(pos_martini[0][0][0]),(pos_martini[0][0][1]),(pos_martini[0][0][2]),1,0))
            martini_counter+=1
            Out_martini_MAP.write("bead"+str(martini_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_martini[3][1])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(martini_masses_sc1)[1:-1]))+"\n")
            martini_final_row = martini_final_row + "bead" + str(martini_counter)  + ","
            Out_martini_PDB.write(pdbAtomLine%(martini_counter,"SC1",resname,"",resi,chr(insc),(pos_martini[0][1][0]),(pos_martini[0][1][1]),(pos_martini[0][1][2]),1,0))                    
        elif len(map_martini) == 3:
            for d in range(0, len(map_martini[0])):
                martini_masses_bb.append(map_martini[0][d][0])
            for g in range(0, len(map_martini[1])):
                martini_masses_sc1.append(map_martini[1][g][0])
            for h in range(0, len(map_martini[2])):
                martini_masses_sc2.append(map_martini[2][h][0])
            martini_counter+=1
            Out_martini_MAP.write("bead"+str(martini_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_martini[3][0])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(martini_masses_bb)[1:-1]))+"\n")
            martini_final_row = martini_final_row + "bead" + str(martini_counter)  + ","
            Out_martini_PDB.write(pdbAtomLine%(martini_counter,"BB",resname,"",resi,chr(insc),(pos_martini[0][0][0]),(pos_martini[0][0][1]),(pos_martini[0][0][2]),1,0))
            martini_counter+=1
            Out_martini_MAP.write("bead"+str(martini_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_martini[3][1])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(martini_masses_sc1)[1:-1]))+"\n")
            martini_final_row = martini_final_row + "bead" + str(martini_counter)  + ","
            Out_martini_PDB.write(pdbAtomLine%(martini_counter,"SC1",resname,"",resi,chr(insc),(pos_martini[0][1][0]),(pos_martini[0][1][1]),(pos_martini[0][1][2]),1,0))                                    
            martini_counter+=1
            Out_martini_MAP.write("bead"+str(martini_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_martini[3][2])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(martini_masses_sc2)[1:-1]))+"\n")
            martini_final_row = martini_final_row + "bead" + str(martini_counter)  + ","
            Out_martini_PDB.write(pdbAtomLine%(martini_counter,"SC2",resname,"",resi,chr(insc),(pos_martini[0][2][0]),(pos_martini[0][2][1]),(pos_martini[0][2][2]),1,0))                                    
        elif len(map_martini) == 4:
            for d in range(0, len(map_martini[0])):
                martini_masses_bb.append(map_martini[0][d][0])
            for g in range(0, len(map_martini[1])):
                martini_masses_sc1.append(map_martini[1][g][0])
            for h in range(0, len(map_martini[2])):
                martini_masses_sc2.append(map_martini[2][h][0])
            for k in range(0, len(map_martini[3])):
                martini_masses_sc3.append(map_martini[3][k][0])
            martini_counter+=1
            Out_martini_MAP.write("bead"+str(martini_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_martini[3][0])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(martini_masses_bb)[1:-1]))+"\n")
            martini_final_row = martini_final_row + "bead" + str(martini_counter)  + ","
            Out_martini_PDB.write(pdbAtomLine%(martini_counter,"BB",resname,"",resi,chr(insc),(pos_martini[0][0][0]),(pos_martini[0][0][1]),(pos_martini[0][0][2]),1,0))
            martini_counter+=1
            Out_martini_MAP.write("bead"+str(martini_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_martini[3][1])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(martini_masses_sc1)[1:-1]))+"\n")
            martini_final_row = martini_final_row + "bead" + str(martini_counter)  + ","
            Out_martini_PDB.write(pdbAtomLine%(martini_counter,"SC1",resname,"",resi,chr(insc),(pos_martini[0][1][0]),(pos_martini[0][1][1]),(pos_martini[0][1][2]),1,0)) 
            martini_counter+=1
            Out_martini_MAP.write("bead"+str(martini_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_martini[3][2])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(martini_masses_sc2)[1:-1]))+"\n")
            martini_final_row = martini_final_row + "bead" + str(martini_counter)  + ","
            Out_martini_PDB.write(pdbAtomLine%(martini_counter,"SC2",resname,"",resi,chr(insc),(pos_martini[0][2][0]),(pos_martini[0][2][1]),(pos_martini[0][2][2]),1,0))                                    
            martini_counter+=1
            Out_martini_MAP.write("bead"+str(martini_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_martini[3][3])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(martini_masses_sc3)[1:-1]))+"\n")
            martini_final_row = martini_final_row + "bead" + str(martini_counter)  + ","  
            Out_martini_PDB.write(pdbAtomLine%(martini_counter,"SC3",resname,"",resi,chr(insc),(pos_martini[0][3][0]),(pos_martini[0][3][1]),(pos_martini[0][3][2]),1,0))                                    
        elif len(map_martini) == 5:
            for d in range(0, len(map_martini[0])):
                martini_masses_bb.append(map_martini[0][d][0])
            for g in range(0, len(map_martini[1])):
                martini_masses_sc1.append(map_martini[1][g][0])
            for h in range(0, len(map_martini[2])):
                martini_masses_sc2.append(map_martini[2][h][0])
            for k in range(0, len(map_martini[3])):
                martini_masses_sc3.append(map_martini[3][k][0])
            for l in range(0, len(map_martini[4])):
                martini_masses_sc4.append(map_martini[4][l][0])
            martini_counter+=1
            Out_martini_MAP.write("bead"+str(martini_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_martini[3][0])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(martini_masses_bb)[1:-1]))+"\n")
            martini_final_row = martini_final_row + "bead" + str(martini_counter)  + ","
            Out_martini_PDB.write(pdbAtomLine%(martini_counter,"BB",resname,"",resi,chr(insc),(pos_martini[0][0][0]),(pos_martini[0][0][1]),(pos_martini[0][0][2]),1,0))
            martini_counter+=1
            Out_martini_MAP.write("bead"+str(martini_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_martini[3][1])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(martini_masses_sc1)[1:-1]))+"\n")
            martini_final_row = martini_final_row + "bead" + str(martini_counter)  + "," 
            Out_martini_PDB.write(pdbAtomLine%(martini_counter,"SC1",resname,"",resi,chr(insc),(pos_martini[0][1][0]),(pos_martini[0][1][1]),(pos_martini[0][1][2]),1,0)) 
            martini_counter+=1
            Out_martini_MAP.write("bead"+str(martini_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_martini[3][2])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(martini_masses_sc2)[1:-1]))+"\n")
            martini_final_row = martini_final_row + "bead" + str(martini_counter)  + ","  
            Out_martini_PDB.write(pdbAtomLine%(martini_counter,"SC2",resname,"",resi,chr(insc),(pos_martini[0][2][0]),(pos_martini[0][2][1]),(pos_martini[0][2][2]),1,0))                                    
            martini_counter+=1
            Out_martini_MAP.write("bead"+str(martini_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_martini[3][3])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(martini_masses_sc3)[1:-1]))+"\n")
            martini_final_row = martini_final_row + "bead" + str(martini_counter)  + ","  
            Out_martini_PDB.write(pdbAtomLine%(martini_counter,"SC3",resname,"",resi,chr(insc),(pos_martini[0][3][0]),(pos_martini[0][3][1]),(pos_martini[0][3][2]),1,0))                                    
            martini_counter+=1
            Out_martini_MAP.write("bead"+str(martini_counter)+": CENTER NOPBC ATOMS="+(clean_whitespaces(str(pos_martini[3][4])[1:-1]))+" WEIGHTS="+(clean_whitespaces(str(martini_masses_sc4)[1:-1]))+"\n")
            martini_final_row = martini_final_row + "bead" + str(martini_counter)  + "," 
            Out_martini_PDB.write(pdbAtomLine%(martini_counter,"SC4",resname,"",resi,chr(insc),(pos_martini[0][4][0]),(pos_martini[0][4][1]),(pos_martini[0][4][2]),1,0))    

number_of_beads_mono=mono_counter
number_of_beads_dual=dual_counter
number_of_beads_martini=martini_counter          
    

if MONO_CG_switch == True:
    Out_1B_MAP.write("\n"+(str(mono_final_row))[:-1])
    Out_1B_MAP.close()
    Out_1B_PDB.close()
    file_checker("plumed_1B.dat","template_1B.pdb")

if DUAL_CG_switch == True:    
    Out_2B_MAP.write("\n"+(str(dual_final_row))[:-1])
    Out_2B_MAP.close()
    Out_2B_PDB.close()
    file_checker("plumed_2B.dat","template_2B.pdb")

if MARTINI_CG_switch == True:
    Out_martini_MAP.write("\n"+(str(martini_final_row))[:-1])
    Out_martini_MAP.close()
    Out_martini_PDB.close()
    file_checker("plumed_martini.dat","template_martini.pdb")

MAIN - PDBs BEAD FORM FACTOR

In [ ]:
############ DICTIONARIES ##########
sum_1B = {}; sum_2B = {}; sum_Martini = {};
sqs_1B = {}; sqs_2B = {}; sqs_Martini = {};

aver_1B = {}; dev_1B = {}; ff0_1B={};
aver_2B = {}; dev_2B = {}; ff0_2B={};
aver_Martini = {}; dev_Martini = {}; ff0_Martini={};
#####################################

# Atomic Form Factor for different qval
atomicff = { }
atomicff['H']=FormFactor('H',qval)
atomicff['C']=FormFactor('C',qval)
atomicff['N']=FormFactor('N',qval)
atomicff['O']=FormFactor('O',qval)
atomicff['P']=FormFactor('P',qval)
atomicff['S']=FormFactor('S',qval)

mono_counter=0; dual_counter=0; martini_counter=0;
map_1b=[]; map_2b=[]; map_martini=[];
       
for pdbid in range(len(ListOfPDB)):
    logging.info("Analysing pdb-id %d" %pdbid)
    c=ListOfPDB[pdbid] 

    for i in order:
        ci = c[i]
        n=0
        for residue,resname in zip(ci.residues,ci.sequence):
            resi = residue[0][2]
            insc  = resi >>20
            resi -= insc<<20
            outname_root="chain"+str(i)+"_"+str(resi)+"_"+resname       #outname_root identifies the residue, used then to store average, standard deviation, q factor @ 0

#### ONE_BEAD ####          
            if MONO_FF_switch==True:
                map_1b=mapMono(residue)
                bff=beadff(map_1b[0],qval,atomicff)
                if outname_root in sum_1B.keys(): 
                    sum_1B[outname_root]+=bff; 
                    sqs_1B[outname_root]+=np.power(bff,2); 
                else: 
                    sum_1B[outname_root]=bff;
                    sqs_1B[outname_root]=np.power(bff,2);
                if debug_FF==True: print_beadff(bff,"1B_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+".ff")     
    
                if pdbid == (len(ListOfPDB)-1):
                    if resi == 1:
                        logging.info("Averaging Form Factors.")
                    mono_counter+=1
                    aver_1B[outname_root]=sum_1B[outname_root]/len(ListOfPDB);
                    dev_1B[outname_root]=np.sqrt(np.round(sqs_1B[outname_root]/len(ListOfPDB)-np.power(aver_1B[outname_root],2),10));
                    ff0_1B[outname_root]=beadff0(map_1b[0],atomicff)
                    if debug_FF0 == True:
                        print_beadff00("1B_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(mono_counter,number_of_beads_mono)+"_"+outname_root+".ff",ff0_1B[outname_root])
                    if debug_AVER==True:
                        print_beadff(aver_1B[outname_root],"1B_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(mono_counter,number_of_beads_mono)+"_"+outname_root+".ff")
                        print_beadff(dev_1B[outname_root],"1B_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(mono_counter,number_of_beads_mono)+"_"+outname_root+".ff")

#### DUAL_BEAD ####
            if DUAL_FF_switch==True:
                map_2b=mapDual(residue)
                bff=beadff(map_2b[0],qval,atomicff)
                if outname_root+"_bb" in sum_2B.keys(): 
                    sum_2B[outname_root+"_bb"]+=bff; 
                    sqs_2B[outname_root+"_bb"]+=np.power(bff,2); 
                else: 
                    sum_2B[outname_root+"_bb"]=bff;
                    sqs_2B[outname_root+"_bb"]=np.power(bff,2); 
                if debug_FF==True: print_beadff(bff,"2B_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_bb.ff")

                if resname != "GLY":
                    # gly does not have sc
                    bff=beadff(map_2b[1],qval,atomicff)
                    if outname_root+"_sc" in sum_2B.keys(): 
                        sum_2B[outname_root+"_sc"]+=bff;
                        sqs_2B[outname_root+"_sc"]+=np.power(bff,2); 
                    else: 
                        sum_2B[outname_root+"_sc"]=bff;
                        sqs_2B[outname_root+"_sc"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"2B_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_sc.ff")

                if pdbid == (len(ListOfPDB)-1):
                    dual_counter+=1
                    aver_2B[outname_root+"_bb"]=sum_2B[outname_root+"_bb"]/len(ListOfPDB);
                    dev_2B[outname_root+"_bb"]=np.sqrt(np.round(sqs_2B[outname_root+"_bb"]/len(ListOfPDB)-np.power(aver_2B[outname_root+"_bb"],2),10));
                    ff0_2B[outname_root+"_bb"]=beadff0(map_2b[0],atomicff)
                    if debug_FF0 == True:
                        print_beadff00("2B_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(dual_counter,number_of_beads_dual)+"_"+outname_root+"_bb.ff",ff0_2B[outname_root+"_bb"])                    
                    if debug_AVER==True:
                        print_beadff(aver_2B[outname_root+"_bb"],"2B_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(dual_counter,number_of_beads_dual)+"_"+outname_root+"_bb.ff")
                        print_beadff(dev_2B[outname_root+"_bb"],"2B_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(dual_counter,number_of_beads_dual)+"_"+outname_root+"_bb.ff")
                    if resname != "GLY":
                        dual_counter+=1
                        aver_2B[outname_root+"_sc"]=sum_2B[outname_root+"_sc"]/len(ListOfPDB);
                        dev_2B[outname_root+"_sc"]=np.sqrt(np.round(sqs_2B[outname_root+"_sc"]/len(ListOfPDB)-np.power(aver_2B[outname_root+"_sc"],2),10));
                        ff0_2B[outname_root+"_sc"]=beadff0(map_2b[1],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("2B_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(dual_counter,number_of_beads_dual)+"_"+outname_root+"_sc.ff",ff0_2B[outname_root+"_sc"])                         
                        if debug_AVER==True:
                            print_beadff(aver_2B[outname_root+"_sc"],"2B_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(dual_counter,number_of_beads_dual)+"_"+outname_root+"_sc.ff")
                            print_beadff(dev_2B[outname_root+"_sc"],"2B_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(dual_counter,number_of_beads_dual)+"_"+outname_root+"_sc.ff")           
  
#### MARTINI_BEAD ####
            if MARTINI_FF_switch==True:
                map_martini=mapMartini(residue)

                if len(map_martini) == 1:
                    bff=beadff(map_martini[0],qval,atomicff)
                    if outname_root+"_bb" in sum_Martini.keys(): 
                        sum_Martini[outname_root+"_bb"]+=bff; 
                        sqs_Martini[outname_root+"_bb"]+=np.power(bff,2); 
                    else: 
                        sum_Martini[outname_root+"_bb"]=bff;
                        sqs_Martini[outname_root+"_bb"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"martini_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_bb.ff")
  
                elif len(map_martini) == 2:
                    bff=beadff(map_martini[0],qval,atomicff)
                    if outname_root+"_bb" in sum_Martini.keys(): 
                        sum_Martini[outname_root+"_bb"]+=bff; 
                        sqs_Martini[outname_root+"_bb"]+=np.power(bff,2); 
                    else: 
                        sum_Martini[outname_root+"_bb"]=bff;
                        sqs_Martini[outname_root+"_bb"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"martini_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_bb.ff")
    
                    bff=beadff(map_martini[1],qval,atomicff)
                    if outname_root+"_sc1" in sum_Martini.keys(): 
                        sum_Martini[outname_root+"_sc1"]+=bff; 
                        sqs_Martini[outname_root+"_sc1"]+=np.power(bff,2); 
                    else: 
                        sum_Martini[outname_root+"_sc1"]=bff;
                        sqs_Martini[outname_root+"_sc1"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"martini_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_sc1.ff")       
    
                elif len(map_martini) == 3:
                    bff=beadff(map_martini[0],qval,atomicff)
                    if outname_root+"_bb" in sum_Martini.keys(): 
                        sum_Martini[outname_root+"_bb"]+=bff; 
                        sqs_Martini[outname_root+"_bb"]+=np.power(bff,2); 
                    else: 
                        sum_Martini[outname_root+"_bb"]=bff;
                        sqs_Martini[outname_root+"_bb"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"martini_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_bb.ff")
    
                    bff=beadff(map_martini[1],qval,atomicff)
                    if outname_root+"_sc1" in sum_Martini.keys(): 
                        sum_Martini[outname_root+"_sc1"]+=bff; 
                        sqs_Martini[outname_root+"_sc1"]+=np.power(bff,2); 
                    else: 
                        sum_Martini[outname_root+"_sc1"]=bff;
                        sqs_Martini[outname_root+"_sc1"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"martini_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_sc1.ff")            
    
                    bff=beadff(map_martini[2],qval,atomicff)
                    if outname_root+"_sc2" in sum_Martini.keys(): 
                        sum_Martini[outname_root+"_sc2"]+=bff; 
                        sqs_Martini[outname_root+"_sc2"]+=np.power(bff,2); 
                    else: 
                        sum_Martini[outname_root+"_sc2"]=bff;
                        sqs_Martini[outname_root+"_sc2"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"martini_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_sc2.ff")    
    
                elif len(map_martini) == 4:
                    bff=beadff(map_martini[0],qval,atomicff)
                    if outname_root+"_bb" in sum_Martini.keys(): 
                        sum_Martini[outname_root+"_bb"]+=bff; 
                        sqs_Martini[outname_root+"_bb"]+=np.power(bff,2); 
                    else: 
                        sum_Martini[outname_root+"_bb"]=bff;
                        sqs_Martini[outname_root+"_bb"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"martini_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_bb.ff")
    
                    bff=beadff(map_martini[1],qval,atomicff)
                    if outname_root+"_sc1" in sum_Martini.keys(): 
                        sum_Martini[outname_root+"_sc1"]+=bff; 
                        sqs_Martini[outname_root+"_sc1"]+=np.power(bff,2); 
                    else: 
                        sum_Martini[outname_root+"_sc1"]=bff;
                        sqs_Martini[outname_root+"_sc1"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"martini_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_sc1.ff")            
    
                    bff=beadff(map_martini[2],qval,atomicff)
                    if outname_root+"_sc2" in sum_Martini.keys(): 
                        sum_Martini[outname_root+"_sc2"]+=bff; 
                        sqs_Martini[outname_root+"_sc2"]+=np.power(bff,2); 
                    else: 
                        sum_Martini[outname_root+"_sc2"]=bff;
                        sqs_Martini[outname_root+"_sc2"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"martini_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_sc2.ff")                                    
    
                    bff=beadff(map_martini[3],qval,atomicff)
                    if outname_root+"_sc3" in sum_Martini.keys(): 
                        sum_Martini[outname_root+"_sc3"]+=bff; 
                        sqs_Martini[outname_root+"_sc3"]+=np.power(bff,2); 
                    else: 
                        sum_Martini[outname_root+"_sc3"]=bff;
                        sqs_Martini[outname_root+"_sc3"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"martini_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_sc3.ff")     
    
                elif len(map_martini) == 5:
                    bff=beadff(map_martini[0],qval,atomicff)
                    if outname_root+"_bb" in sum_Martini.keys(): 
                        sum_Martini[outname_root+"_bb"]+=bff; 
                        sqs_Martini[outname_root+"_bb"]+=np.power(bff,2); 
                    else: 
                        sum_Martini[outname_root+"_bb"]=bff;
                        sqs_Martini[outname_root+"_bb"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"martini_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_bb.ff")
    
                    bff=beadff(map_martini[1],qval,atomicff)
                    if outname_root+"_sc1" in sum_Martini.keys(): 
                        sum_Martini[outname_root+"_sc1"]+=bff; 
                        sqs_Martini[outname_root+"_sc1"]+=np.power(bff,2); 
                    else: 
                        sum_Martini[outname_root+"_sc1"]=bff;
                        sqs_Martini[outname_root+"_sc1"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"martini_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_sc1.ff")            
    
                    bff=beadff(map_martini[2],qval,atomicff)
                    if outname_root+"_sc2" in sum_Martini.keys(): 
                        sum_Martini[outname_root+"_sc2"]+=bff; 
                        sqs_Martini[outname_root+"_sc2"]+=np.power(bff,2); 
                    else: 
                        sum_Martini[outname_root+"_sc2"]=bff;
                        sqs_Martini[outname_root+"_sc2"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"martini_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_sc2.ff")                                    
    
                    bff=beadff(map_martini[3],qval,atomicff)
                    if outname_root+"_sc3" in sum_Martini.keys(): 
                        sum_Martini[outname_root+"_sc3"]+=bff; 
                        sqs_Martini[outname_root+"_sc3"]+=np.power(bff,2); 
                    else: 
                        sum_Martini[outname_root+"_sc3"]=bff;
                        sqs_Martini[outname_root+"_sc3"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"martini_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_sc3.ff")
    
                    bff=beadff(map_martini[4],qval,atomicff)
                    if outname_root+"_sc4" in sum_Martini.keys(): 
                        sum_Martini[outname_root+"_sc4"]+=bff; 
                        sqs_Martini[outname_root+"_sc4"]+=np.power(bff,2); 
                    else: 
                        sum_Martini[outname_root+"_sc4"]=bff;
                        sqs_Martini[outname_root+"_sc4"]=np.power(bff,2); 
                    if debug_FF==True: print_beadff(bff,"martini_"+(str(get_file_names.number_of_pdbs))+"/"+outname_root+"_sc4.ff")
                    
                if pdbid == (len(ListOfPDB)-1):
                    if len(map_martini) == 1:
                        martini_counter+=1
                        aver_Martini[outname_root+"_bb"]=sum_Martini[outname_root+"_bb"]/len(ListOfPDB);
                        dev_Martini[outname_root+"_bb"]=np.sqrt(np.round(sqs_Martini[outname_root+"_bb"]/len(ListOfPDB)-np.power(aver_Martini[outname_root+"_bb"],2),10));
                        ff0_Martini[outname_root+"_bb"]=beadff0(map_martini[0],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("martini_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_bb.ff",ff0_Martini[outname_root+"_bb"])                       
                        if debug_AVER==True:
                            print_beadff(aver_Martini[outname_root+"_bb"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_bb.ff")
                            print_beadff(dev_Martini[outname_root+"_bb"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_bb.ff")
                    elif len(map_martini) == 2:
                        martini_counter+=1
                        aver_Martini[outname_root+"_bb"]=sum_Martini[outname_root+"_bb"]/len(ListOfPDB);
                        dev_Martini[outname_root+"_bb"]=np.sqrt(np.round(sqs_Martini[outname_root+"_bb"]/len(ListOfPDB)-np.power(aver_Martini[outname_root+"_bb"],2),10));
                        ff0_Martini[outname_root+"_bb"]=beadff0(map_martini[0],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("martini_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_bb.ff",ff0_Martini[outname_root+"_bb"])                       
                        if debug_AVER==True:
                            print_beadff(aver_Martini[outname_root+"_bb"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_bb.ff")
                            print_beadff(dev_Martini[outname_root+"_bb"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_bb.ff")
                        martini_counter+=1                                              
                        aver_Martini[outname_root+"_sc1"]=sum_Martini[outname_root+"_sc1"]/len(ListOfPDB);
                        dev_Martini[outname_root+"_sc1"]=np.sqrt(np.round(sqs_Martini[outname_root+"_sc1"]/len(ListOfPDB)-np.power(aver_Martini[outname_root+"_sc1"],2),10));
                        ff0_Martini[outname_root+"_sc1"]=beadff0(map_martini[1],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("martini_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc1.ff",ff0_Martini[outname_root+"_sc1"])                        
                        if debug_AVER==True:
                            print_beadff(aver_Martini[outname_root+"_sc1"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc1.ff")
                            print_beadff(dev_Martini[outname_root+"_sc1"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc1.ff")                                 
                    elif len(map_martini) == 3:
                        martini_counter+=1
                        aver_Martini[outname_root+"_bb"]=sum_Martini[outname_root+"_bb"]/len(ListOfPDB);
                        dev_Martini[outname_root+"_bb"]=np.sqrt(np.round(sqs_Martini[outname_root+"_bb"]/len(ListOfPDB)-np.power(aver_Martini[outname_root+"_bb"],2),10));
                        ff0_Martini[outname_root+"_bb"]=beadff0(map_martini[0],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("martini_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_bb.ff",ff0_Martini[outname_root+"_bb"])                       
                        if debug_AVER==True:                        
                            print_beadff(aver_Martini[outname_root+"_bb"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_bb.ff")
                            print_beadff(dev_Martini[outname_root+"_bb"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_bb.ff")
                        martini_counter+=1                                              
                        aver_Martini[outname_root+"_sc1"]=sum_Martini[outname_root+"_sc1"]/len(ListOfPDB);
                        dev_Martini[outname_root+"_sc1"]=np.sqrt(np.round(sqs_Martini[outname_root+"_sc1"]/len(ListOfPDB)-np.power(aver_Martini[outname_root+"_sc1"],2),10));
                        ff0_Martini[outname_root+"_sc1"]=beadff0(map_martini[1],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("martini_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc1.ff",ff0_Martini[outname_root+"_sc1"])                        
                        if debug_AVER==True:                        
                            print_beadff(aver_Martini[outname_root+"_sc1"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc1.ff")
                            print_beadff(dev_Martini[outname_root+"_sc1"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc1.ff")
                        martini_counter+=1   
                        aver_Martini[outname_root+"_sc2"]=sum_Martini[outname_root+"_sc2"]/len(ListOfPDB);
                        dev_Martini[outname_root+"_sc2"]=np.sqrt(np.round(sqs_Martini[outname_root+"_sc2"]/len(ListOfPDB)-np.power(aver_Martini[outname_root+"_sc2"],2),10));
                        ff0_Martini[outname_root+"_sc2"]=beadff0(map_martini[2],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("martini_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc2.ff",ff0_Martini[outname_root+"_sc2"])                        
                        if debug_AVER==True:
                            print_beadff(aver_Martini[outname_root+"_sc2"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc2.ff")
                            print_beadff(dev_Martini[outname_root+"_sc2"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc2.ff") 
                    elif len(map_martini) == 4:
                        martini_counter+=1
                        aver_Martini[outname_root+"_bb"]=sum_Martini[outname_root+"_bb"]/len(ListOfPDB);
                        dev_Martini[outname_root+"_bb"]=np.sqrt(np.round(sqs_Martini[outname_root+"_bb"]/len(ListOfPDB)-np.power(aver_Martini[outname_root+"_bb"],2),10));
                        ff0_Martini[outname_root+"_bb"]=beadff0(map_martini[0],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("martini_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_bb.ff",ff0_Martini[outname_root+"_bb"])                       
                        if debug_AVER==True:                        
                            print_beadff(aver_Martini[outname_root+"_bb"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_bb.ff")
                            print_beadff(dev_Martini[outname_root+"_bb"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_bb.ff")
                        martini_counter+=1
                        aver_Martini[outname_root+"_sc1"]=sum_Martini[outname_root+"_sc1"]/len(ListOfPDB);
                        dev_Martini[outname_root+"_sc1"]=np.sqrt(np.round(sqs_Martini[outname_root+"_sc1"]/len(ListOfPDB)-np.power(aver_Martini[outname_root+"_sc1"],2),10));
                        ff0_Martini[outname_root+"_sc1"]=beadff0(map_martini[1],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("martini_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc1.ff",ff0_Martini[outname_root+"_sc1"])                        
                        if debug_AVER==True:                        
                            print_beadff(aver_Martini[outname_root+"_sc1"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc1.ff")
                            print_beadff(dev_Martini[outname_root+"_sc1"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc1.ff")
                        martini_counter+=1   
                        aver_Martini[outname_root+"_sc2"]=sum_Martini[outname_root+"_sc2"]/len(ListOfPDB);
                        dev_Martini[outname_root+"_sc2"]=np.sqrt(np.round(sqs_Martini[outname_root+"_sc2"]/len(ListOfPDB)-np.power(aver_Martini[outname_root+"_sc2"],2),10));
                        ff0_Martini[outname_root+"_sc2"]=beadff0(map_martini[2],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("martini_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc2.ff",ff0_Martini[outname_root+"_sc2"])                        
                        if debug_AVER==True:                        
                            print_beadff(aver_Martini[outname_root+"_sc2"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc2.ff")
                            print_beadff(dev_Martini[outname_root+"_sc2"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc2.ff")
                        martini_counter+=1   
                        aver_Martini[outname_root+"_sc3"]=sum_Martini[outname_root+"_sc3"]/len(ListOfPDB);
                        dev_Martini[outname_root+"_sc3"]=np.sqrt(np.round(sqs_Martini[outname_root+"_sc3"]/len(ListOfPDB)-np.power(aver_Martini[outname_root+"_sc3"],2),10));
                        ff0_Martini[outname_root+"_sc3"]=beadff0(map_martini[3],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("martini_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc3.ff",ff0_Martini[outname_root+"_sc3"])                        
                        if debug_AVER==True:                        
                            print_beadff(aver_Martini[outname_root+"_sc3"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc3.ff")
                            print_beadff(dev_Martini[outname_root+"_sc3"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc3.ff") 
                    elif len(map_martini) == 5:
                        martini_counter+=1
                        aver_Martini[outname_root+"_bb"]=sum_Martini[outname_root+"_bb"]/len(ListOfPDB);
                        dev_Martini[outname_root+"_bb"]=np.sqrt(np.round(sqs_Martini[outname_root+"_bb"]/len(ListOfPDB)-np.power(aver_Martini[outname_root+"_bb"],2),10));
                        ff0_Martini[outname_root+"_bb"]=beadff0(map_martini[0],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("martini_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_bb.ff",ff0_Martini[outname_root+"_bb"])                       
                        if debug_AVER==True:                        
                            print_beadff(aver_Martini[outname_root+"_bb"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_bb.ff")
                            print_beadff(dev_Martini[outname_root+"_bb"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_bb.ff")
                        martini_counter+=1                                              
                        aver_Martini[outname_root+"_sc1"]=sum_Martini[outname_root+"_sc1"]/len(ListOfPDB);
                        dev_Martini[outname_root+"_sc1"]=np.sqrt(np.round(sqs_Martini[outname_root+"_sc1"]/len(ListOfPDB)-np.power(aver_Martini[outname_root+"_sc1"],2),10));
                        ff0_Martini[outname_root+"_sc1"]=beadff0(map_martini[1],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("martini_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc1.ff",ff0_Martini[outname_root+"_sc1"])                        
                        if debug_AVER==True:
                            print_beadff(aver_Martini[outname_root+"_sc1"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc1.ff")
                            print_beadff(dev_Martini[outname_root+"_sc1"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc1.ff")
                        martini_counter+=1   
                        aver_Martini[outname_root+"_sc2"]=sum_Martini[outname_root+"_sc2"]/len(ListOfPDB);
                        dev_Martini[outname_root+"_sc2"]=np.sqrt(np.round(sqs_Martini[outname_root+"_sc2"]/len(ListOfPDB)-np.power(aver_Martini[outname_root+"_sc2"],2),10));
                        ff0_Martini[outname_root+"_sc2"]=beadff0(map_martini[2],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("martini_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc2.ff",ff0_Martini[outname_root+"_sc2"])                        
                        if debug_AVER==True:                        
                            print_beadff(aver_Martini[outname_root+"_sc2"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc2.ff")
                            print_beadff(dev_Martini[outname_root+"_sc2"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc2.ff")   
                        martini_counter+=1
                        aver_Martini[outname_root+"_sc3"]=sum_Martini[outname_root+"_sc3"]/len(ListOfPDB);
                        dev_Martini[outname_root+"_sc3"]=np.sqrt(np.round(sqs_Martini[outname_root+"_sc3"]/len(ListOfPDB)-np.power(aver_Martini[outname_root+"_sc3"],2),10));
                        ff0_Martini[outname_root+"_sc3"]=beadff0(map_martini[3],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("martini_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc3.ff",ff0_Martini[outname_root+"_sc3"])                        
                        if debug_AVER==True:                        
                            print_beadff(aver_Martini[outname_root+"_sc3"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc3.ff")
                            print_beadff(dev_Martini[outname_root+"_sc3"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc3.ff")   
                        martini_counter+=1
                        aver_Martini[outname_root+"_sc4"]=sum_Martini[outname_root+"_sc4"]/len(ListOfPDB);
                        dev_Martini[outname_root+"_sc4"]=np.sqrt(np.round(sqs_Martini[outname_root+"_sc4"]/len(ListOfPDB)-np.power(aver_Martini[outname_root+"_sc4"],2),10));
                        ff0_Martini[outname_root+"_sc4"]=beadff0(map_martini[4],atomicff)
                        if debug_FF0 == True:
                            print_beadff00("martini_ff0_"+(str(get_file_names.number_of_pdbs))+"/ff0_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc4.ff",ff0_Martini[outname_root+"_sc4"])                        
                        if debug_AVER==True:                        
                            print_beadff(aver_Martini[outname_root+"_sc4"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc4.ff")
                            print_beadff(dev_Martini[outname_root+"_sc4"],"martini_"+(str(get_file_names.number_of_pdbs))+"/AVER_"+format_beads_number(martini_counter,number_of_beads_martini)+"_"+outname_root+"_sc4.ff")   

MAIN - PRINTER

In [ ]:
if MONO_FF_switch==True:
    plumed_main_printer("plumed_main_1B.dat","plumed_1B.dat",aver_1B,ff0_1B,"mono")
    file_checker_single("plumed_main_1B.dat")
if DUAL_FF_switch==True:    
    plumed_main_printer("plumed_main_2B.dat","plumed_2B.dat",aver_2B,ff0_2B,"dual")
    file_checker_single("plumed_main_2B.dat")
if MARTINI_FF_switch==True:
    plumed_main_printer("plumed_main_martini.dat","plumed_martini.dat",aver_Martini,ff0_Martini,"martini")
    file_checker_single("plumed_main_martini.dat")

EXTERNAL READER-PRINTER

In [ ]:
if EXTERNAL_FXs == True:

############# SETUP: bead type and path #############
    bead_selector("dual") #mono, dual or martini
    ext_reader_AVER_path=('/mnt/scratch/shared/fede/WORKING/SCRIPT/_working/2B_10/')
    ext_reader_ff0_path=('/mnt/scratch/shared/fede/WORKING/SCRIPT/_working/2B_ff0_10/')
#####################################################
    
    ext_AVER_ff = dict(ext_reader(ext_reader_AVER_path,1))
    ext_ff0 = dict(ext_reader(ext_reader_ff0_path,0))

    plumed_main_printer(bead_selector.plumed_main,bead_selector.plumed_include,ext_AVER_ff,ext_ff0,bead_selector.bead_name)
    file_checker_single(bead_selector.plumed_main)